# Load libraries

In [4]:
library(tidyverse)
library(DESeq2)
library(org.Hs.eg.db)
library(modelr)
setwd("/scratch/s/shreejoy/nxu/trap_seq")

# Load data

In [35]:
dds <- readRDS("proc/dds_wendy.rds")
coldata <- colData(dds) %>% as.data.frame()
rownames(coldata) <- 1:nrow(coldata)

In [40]:
design(dds) <- formula(~ assay + condition + assay:condition)
res_WT <- results(DESeq(dds, test = "LRT", reduced = ~ assay + condition))
colData(dds)$condition <- relevel(colData(dds)$condition, ref="PTEN_KO")
res_KO <- results(DESeq(dds, test = "LRT", reduced = ~ assay + condition))

using pre-existing normalization factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

using pre-existing normalization factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [41]:
res_WT

log2 fold change (MLE): assayInput.conditionPTEN KO 
LRT p-value: '~ assay + condition + assay:condition' vs '~ assay + condition' 
DataFrame with 60136 rows and 6 columns
                   baseMean log2FoldChange     lfcSE        stat    pvalue
                  <numeric>      <numeric> <numeric>   <numeric> <numeric>
EGFP            12439.09054      0.6830110  0.318381 4.59143e+00 0.0321322
ENSG00000000003  1560.44018     -0.9582765  0.294921 1.05156e+01 0.0011837
ENSG00000000005     2.51529      0.2330169  2.448566 9.13523e-03 0.9238554
ENSG00000000419  2052.09944      0.0837046  0.178737 2.19292e-01 0.6395795
ENSG00000000457   738.60995      0.0048639  0.210737 5.32847e-04 0.9815837
...                     ...            ...       ...         ...       ...
ENSG00000288721   285.23702      0.0396225  0.453695  0.00762252  0.930427
ENSG00000288722  1867.83615      0.3066276  0.198373  2.38813983  0.122259
ENSG00000288723     3.47112     -3.0439835  2.081881  2.56589852  0.109191
ENS

In [42]:
res_KO

log2 fold change (MLE): assayInput.conditionWT 
LRT p-value: '~ assay + condition + assay:condition' vs '~ assay + condition' 
DataFrame with 60136 rows and 6 columns
                   baseMean log2FoldChange     lfcSE        stat    pvalue
                  <numeric>      <numeric> <numeric>   <numeric> <numeric>
EGFP            12439.09054     -0.6830118  0.318381 4.59143e+00 0.0321322
ENSG00000000003  1560.44018      0.9582760  0.294921 1.05156e+01 0.0011837
ENSG00000000005     2.51529     -0.2330174  2.448565 9.13523e-03 0.9238554
ENSG00000000419  2052.09944     -0.0837048  0.178737 2.19292e-01 0.6395795
ENSG00000000457   738.60995     -0.0048641  0.210737 5.32847e-04 0.9815837
...                     ...            ...       ...         ...       ...
ENSG00000288721   285.23702     -0.0396232  0.453695  0.00762252  0.930427
ENSG00000288722  1867.83615     -0.3066278  0.198373  2.38813983  0.122259
ENSG00000288723     3.47112      3.0439669  2.081878  2.56589305  0.109191
ENSG0000

# The example starts here

Here the reference level for the factor `condition` is `WT`. The level that comes first is the reference level.

In [17]:
colData(dds)$condition

[1] PTEN_KO PTEN_KO PTEN_KO PTEN_KO WT      WT      WT      WT      WT     
[10] WT      PTEN_KO PTEN_KO
Levels: WT PTEN_KO

As a result of using `WT` as the reference level, the interaction term becomes `assayInput:conditionPTEN_KO`, and it has a value of 1 if the corresponding sample belongs to `assayInput` and `conditionPTEN_KO` at the same time.

In [19]:
model.matrix(~assay + condition + assay:condition, colData(dds))

,(Intercept),assayInput,conditionPTEN_KO,assayInput:conditionPTEN_KO
4_7,1,1,1,1
4_7.1,1,0,1,0
4_9,1,1,1,1
4_9.1,1,0,1,0
87,1,1,0,0
87.1,1,0,0,0
90,1,1,0,0
90.1,1,0,0,0
91,1,1,0,0
91.1,1,0,0,0


We can change the reference level to `PTEN_KO` by:

In [20]:
colData(dds)$condition <- relevel(colData(dds)$condition, "PTEN_KO")

In [22]:
colData(dds)$condition

[1] PTEN_KO PTEN_KO PTEN_KO PTEN_KO WT      WT      WT      WT      WT     
[10] WT      PTEN_KO PTEN_KO
Levels: PTEN_KO WT

As a result of using `PTEN_KO` as the reference level, the interaction term becomes `assayInput:conditionWT`, and has a value of 1 when the corresponding sample belongs to both `assayInput` and `conditionWT`.

In [21]:
model.matrix(~assay + condition + assay:condition, colData(dds))

,(Intercept),assayInput,conditionWT,assayInput:conditionWT
4_7,1,1,0,0
4_7.1,1,0,0,0
4_9,1,1,0,0
4_9.1,1,0,0,0
87,1,1,1,1
87.1,1,0,1,0
90,1,1,1,1
90.1,1,0,1,0
91,1,1,1,1
91.1,1,0,1,0
